In [9]:
from etl.extract.extract_inkipedia import extract_inkipedia
df_weapons = extract_inkipedia()
display(df_weapons)

,Extra_space,Name,ID,Sub,Special,Special_Points,Level,Price,Class,Introduced
0,,Sploosh-o-matic,0,Curling Bomb,Ultra Stamp,180p,9,1,Shooter,Version1.0.0(Drizzle Season 2022)
1,,Neo Sploosh-o-matic,1,Squid Beakon,Killer Wail 5.1,170p,13,1,Shooter,Version3.0.0(Fresh Season 2023)
2,,Splattershot Jr.,10,Splat Bomb,Big Bubbler,180p,1,,Shooter,Version1.0.0(Drizzle Season 2022)
3,,Custom Splattershot Jr.,11,Torpedo,Wave Breaker,190p,6,1,Shooter,Version2.0.0(Chill Season 2022)
4,,Splash-o-matic,20,Burst Bomb,Crab Tank,200p,16,1,Shooter,Version1.0.0(Drizzle Season 2022)
...,...,...,...,...,...,...,...,...,...,...
138,,Order Splatana Replica,8005,Burst Bomb,Zipcaster,210p,1,,Splatana,Version7.0.0(Side Order)
139,,Splatana Wiper,8010,Torpedo,Ultra Stamp,190p,5,1,Splatana,Version1.0.0(Drizzle Season 2022)
140,,Splatana Wiper Deco,8011,Squid Beakon,Tenta Missiles,190p,10,1,Splatana,Version4.0.0(Sizzle Season 2023)
141,,Mint Decavitator,8020,Suction Bomb,Big Bubbler,210p,21,1,Splatana,Version8.0.0(Sizzle Season 2024)


In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from etl.extract.url_request import make_request


SPECIALS_GALLERY_PATH = (
    'https://splatoonwiki.org/wiki/Category:Splatoon_3_special_weapon_icons'
)

# obtain a series of distinct special weapons
special_weapons = df_weapons['Special'].unique()
# create a dataframe of distinct special weapons
special_weapons_df = pd.DataFrame(special_weapons, columns=['Special'])

display(special_weapons_df)

soup = BeautifulSoup(make_request(SPECIALS_GALLERY_PATH).text, "html.parser")

images = soup.find_all('img')

image_urls = []

for img in images:
    src = img.get('src')
    if src and 'Special' in src and 'Rainmaker' not in src:   
        if 'Splashdown' not in src:
            image_urls.append('https:' + src)
        elif 'Triple' in src:
            image_urls.append('https:' + src)

image_urls


len(image_urls)


,Special
0,Ultra Stamp
1,Killer Wail 5.1
2,Big Bubbler
3,Wave Breaker
4,Crab Tank
5,Triple Inkstrike
6,Reefslider
7,Booyah Bomb
8,Trizooka
9,Splattercolor Screen


19

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from etl.extract.url_request import make_request


SPECIALS_GALLERY_PATH = (
    'https://splatoonwiki.org/wiki/Category:Splatoon_3_special_weapon_icons'
)


# function to extract all special weapon images
# from inkipedia as a dataframe
def extract_specials_images(df_weapons):
    print("\nCreating a dataframe of special weapon images...")
    # first call a function that returns all possible special weapons
    # as a dataframe
    special_weapons_df = find_special_names(df_weapons)
    # find all image urls for special images
    image_urls = find_all_special_images()
    # add these to the original specials dataframe
    special_weapons_df = create_special_df(special_weapons_df, image_urls)
    print("Done! " + str(special_weapons_df.shape[0]) + "/19 images found")
    # check the length
    if special_weapons_df.shape[0] == 19:
        # return the completed abilities dataframe
        return special_weapons_df
    else:
        raise Exception(
            "Error: Special weapons data frame only has " +
            str(special_weapons_df.shape[0]) +
            " Specials, some are missing"
        )


# function to find all possible special weapons
def find_special_names(df_weapons):
    # obtain a series of distinct special weapons
    special_weapons = df_weapons['Special'].unique()
    # create a dataframe of distinct special weapons
    special_weapons_df = pd.DataFrame(special_weapons, columns=['Special'])
    
    # return this dataframe of special weapons
    return special_weapons_df


# function to find all special weapon images
def find_all_special_images():
    # make request to the site
    soup = BeautifulSoup(make_request(SPECIALS_GALLERY_PATH).text, "html.parser")
    # obtain all images
    images = soup.find_all('img')
    image_urls = []  # to store urls
    # loop through images on the page
    for img in images:
        # call a function to add the image url to the list
        image_urls = find_special_image(img, image_urls)
    # return the complete list
    return image_urls

# function to check if the image is a special
# if so add to the list or urls
def find_special_image(img, image_urls):
    src = img.get('src')
    if src and 'Special' in src and 'Rainmaker' not in src:   
        if 'Splashdown' not in src:
            image_urls.append('https:' + src)
        elif 'Triple' in src:
            image_urls.append('https:' + src)
    return image_urls


# function to use both the list of images
# and dataframe of weapon names
# to create a complete dataframe of ability images
def create_special_df(special_weapons_df, image_urls):
    # first order the specials alphabetically
    special_weapons_df = special_weapons_df.sort_values(by='Special')
    # reset the index again
    special_weapons_df.reset_index(drop=True, inplace=True)
    # --- Assistance from ChatGPT ---------------------------------------
    # order the specials
    image_urls = sorted(image_urls, key=lambda img: img.split('/')[-1])
    # -------------------------------------------------------------------
    # add a new column called Image URL
    # add the urls here
    special_weapons_df['Image URL'] = image_urls
    # return the complete dataframe
    return special_weapons_df

In [12]:
extract_specials_images(df_weapons)


Creating a dataframe of special weapon images...
Done! 19/19 images found


,Special,Image URL
0,Big Bubbler,https://cdn.wikimg.net/en/splatoonwiki/images/...
1,Booyah Bomb,https://cdn.wikimg.net/en/splatoonwiki/images/...
2,Crab Tank,https://cdn.wikimg.net/en/splatoonwiki/images/...
3,Ink Storm,https://cdn.wikimg.net/en/splatoonwiki/images/...
4,Ink Vac,https://cdn.wikimg.net/en/splatoonwiki/images/...
5,Inkjet,https://cdn.wikimg.net/en/splatoonwiki/images/...
6,Killer Wail 5.1,https://cdn.wikimg.net/en/splatoonwiki/images/...
7,Kraken Royale,https://cdn.wikimg.net/en/splatoonwiki/images/...
8,Reefslider,https://cdn.wikimg.net/en/splatoonwiki/images/...
9,Splattercolor Screen,https://cdn.wikimg.net/en/splatoonwiki/images/...


# Complete with functions

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from etl.extract.url_request import make_request
import time

SPECIALS_GALLERY_PATH = (
    'https://splatoonwiki.org/wiki/Category:Splatoon_3_special_weapon_icons'
)


# function to extract all special weapon images
# from inkipedia as a dataframe
def extract_specials_images(df_weapons):
    print("\nCreating a dataframe of special weapon images...")
    # first call a function that returns all possible special weapons
    # as a dataframe
    special_weapons_df = find_special_names(df_weapons)
    # find all image urls for special images
    image_urls = find_all_special_images()
    # add these to the original specials dataframe
    special_weapons_df = create_special_df(special_weapons_df, image_urls)
    print("Done! " + str(special_weapons_df.shape[0]) + "/19 images found")
    # check the length
    if special_weapons_df.shape[0] == 19:
        # return the completed abilities dataframe
        return special_weapons_df
    else:
        raise Exception(
            "Error: Special weapons data frame only has " +
            str(special_weapons_df.shape[0]) +
            " Specials, some are missing"
        )


# function to find all possible special weapons
def find_special_names(df_weapons):
    # obtain a series of distinct special weapons
    special_weapons = df_weapons['Special'].unique()
    # create a dataframe of distinct special weapons
    special_weapons_df = pd.DataFrame(special_weapons, columns=['Special'])
    # return this dataframe of special weapons
    return special_weapons_df


# function to find all special weapon images
def find_all_special_images():
    # make request to the site
    soup = BeautifulSoup(
        make_request(SPECIALS_GALLERY_PATH).text, "html.parser"
        )
    # obtain all images
    images = soup.find_all('img')
    image_urls = []  # to store urls
    # pause
    time.sleep(1)
    # loop through images on the page
    for img in images:
        # pause
        time.sleep(0.05)
        # call a function to add the image url to the list
        image_urls = find_special_image(img, image_urls)
    # return the complete list
    return image_urls


# function to check if the image is a special
# if so add to the list or urls
def find_special_image(img, image_urls):
    src = img.get('src')
    if src and 'Special' in src and 'Rainmaker' not in src:
        if 'Splashdown' not in src:
            image_urls.append('https:' + src)
        elif 'Triple' in src:
            image_urls.append('https:' + src)
    return image_urls


# function to use both the list of images
# and dataframe of weapon names
# to create a complete dataframe of ability images
def create_special_df(special_weapons_df, image_urls):
    # first order the specials alphabetically
    special_weapons_df = special_weapons_df.sort_values(by='Special')
    # reset the index again
    special_weapons_df.reset_index(drop=True, inplace=True)
    # --- Assistance from ChatGPT ---------------------------------------
    # order the specials
    image_urls = sorted(image_urls, key=lambda img: img.split('/')[-1])
    # -------------------------------------------------------------------
    # add a new column called Image URL
    # add the urls here
    special_weapons_df['Image URL'] = image_urls
    # return the complete dataframe
    return special_weapons_df




<img height="32" src="//cdn.wikimg.net/en/splatoonwiki/images/7/7f/Site-wordmark.svg" width="152"/>




<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/e/ef/S3_Weapon_Special_Big_Bubbler.png/120px-S3_Weapon_Special_Big_Bubbler.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/e/ef/S3_Weapon_Special_Big_Bubbler.png 1.5x" width="120"/>

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/2/23/S3_Weapon_Special_Splashdown.png/120px-S3_Weapon_Special_Splashdown.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/2/23/S3_Weapon_Special_Splashdown.png 1.5x" width="120"/>

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/b/b8/S3_Weapon_Special_Rainmaker.png/120px-S3_Weapon_Special_Rainmaker.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/b/b8/S3_Weapon_Special_Rainmaker.png 1.5x" width="120"/>

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/b/b3/S3_Weapon_Special_Triple_Splashdown.png/120px-S3_Weapon_Special_Triple_Splashdown.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/b/b3/S3_Weapon_Special_Triple_Splashdown.png 1.5x" width="120"/>

In [28]:
extract_specials_images(df_weapons)


Creating a dataframe of special weapon images...

hello

<img height="32" src="//cdn.wikimg.net/en/splatoonwiki/images/7/7f/Site-wordmark.svg" width="152"/>

hello

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/e/ef/S3_Weapon_Special_Big_Bubbler.png/120px-S3_Weapon_Special_Big_Bubbler.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/e/ef/S3_Weapon_Special_Big_Bubbler.png 1.5x" width="120"/>

hello

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/0/00/S3_Weapon_Special_Booyah_Bomb.png/120px-S3_Weapon_Special_Booyah_Bomb.png" srcset="//cdn.wikimg.net/en/splatoonwiki/images/0/00/S3_Weapon_Special_Booyah_Bomb.png 1.5x" width="120"/>

hello

<img alt="" decoding="async" height="120" loading="lazy" src="//cdn.wikimg.net/en/splatoonwiki/images/thumb/8/88/S3_Weapon_Special_Crab_Tank.png/120px-S3_Weapon_Special_Crab_Tank.png" srcset="//cdn.wikimg.net/en/splatoonwiki/ima

,Special,Image URL
0,Big Bubbler,https://cdn.wikimg.net/en/splatoonwiki/images/...
1,Booyah Bomb,https://cdn.wikimg.net/en/splatoonwiki/images/...
2,Crab Tank,https://cdn.wikimg.net/en/splatoonwiki/images/...
3,Ink Storm,https://cdn.wikimg.net/en/splatoonwiki/images/...
4,Ink Vac,https://cdn.wikimg.net/en/splatoonwiki/images/...
5,Inkjet,https://cdn.wikimg.net/en/splatoonwiki/images/...
6,Killer Wail 5.1,https://cdn.wikimg.net/en/splatoonwiki/images/...
7,Kraken Royale,https://cdn.wikimg.net/en/splatoonwiki/images/...
8,Reefslider,https://cdn.wikimg.net/en/splatoonwiki/images/...
9,Splattercolor Screen,https://cdn.wikimg.net/en/splatoonwiki/images/...
